In [1]:
import glfw
from OpenGL.GL import *
from OpenGL.GLU import *
import numpy as np
import glob
import csv

# ---------- SETTINGS ---------- #
trail_length = 100
frame_delay = 0.01  # seconds between frames

colors = [
    (1, 1, 0),  # Sun - Yellow
    (0.5, 0.5, 0.5),  # Mercury - Gray
    (1, 0.6, 0),  # Venus - Orange
    (0, 0, 1),  # Earth - Blue
    (1, 0, 0),  # Mars - Red
    (1, 1, 1),  # Jupiter - White
    (1, 0.5, 1),  # Saturn - Pink
    (0, 1, 1),  # Uranus - Cyan
    (0.5, 0, 1),  # Neptune - Purple
]

body_names = ["Sun", "Mercury", "Venus", "Earth", "Mars", "Jupiter", "Saturn", "Uranus", "Neptune"]

# ---------- LOAD DATA ---------- #
def load_data():
    positions = []
    max_val = 0
    for name in body_names:
        data = np.genfromtxt(f"{name}.csv", delimiter=",", skip_header=1)
        xyz = data[:, 1:4]  # X, Y, Z
        max_val = max(max_val, np.max(np.abs(xyz)))
        positions.append(xyz)
    return np.array(positions), max_val

positions, max_abs = load_data()
num_bodies, num_frames, _ = positions.shape

# ---------- SCALE DATA ---------- #
SCALE = 100.0 / max_abs
positions *= SCALE

# ---------- TRAIL SETUP ---------- #
trails = [[] for _ in range(num_bodies)]
frame = 0

# ---------- DRAW AXES ---------- #
def draw_axes():
    glBegin(GL_LINES)
    glColor3f(1, 0, 0)  # X
    glVertex3f(-100, 0, 0)
    glVertex3f(100, 0, 0)

    glColor3f(0, 1, 0)  # Y
    glVertex3f(0, -100, 0)
    glVertex3f(0, 100, 0)

    glColor3f(0, 0, 1)  # Z
    glVertex3f(0, 0, -100)
    glVertex3f(0, 0, 100)
    glEnd()

# ---------- DRAW TRAILS ---------- #
def draw_trails():
    for i in range(num_bodies):
        glColor3f(*colors[i % len(colors)])
        glBegin(GL_LINE_STRIP)
        for point in trails[i]:
            glVertex3f(*point)
        glEnd()

# ---------- MAIN ---------- #
if not glfw.init():
    raise Exception("GLFW can't be initialized")

window = glfw.create_window(1200, 900, "N-Body 3D Simulation", None, None)
if not window:
    glfw.terminate()
    raise Exception("GLFW window creation failed")

glfw.make_context_current(window)
glEnable(GL_DEPTH_TEST)

while not glfw.window_should_close(window):
    glfw.poll_events()
    glClearColor(0.05, 0.05, 0.15, 1)
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)

    # Projection + View
    glMatrixMode(GL_PROJECTION)
    glLoadIdentity()
    gluPerspective(45, 1200 / 900, 0.1, 500)
    glMatrixMode(GL_MODELVIEW)
    glLoadIdentity()

    # Camera orbit animation
    angle = frame * 0.5
    radius = 200
    cam_x = np.cos(np.radians(angle)) * radius
    cam_z = np.sin(np.radians(angle)) * radius
    gluLookAt(cam_x, 100, cam_z, 0, 0, 0, 0, 1, 0)

    draw_axes()

    if frame < num_frames:
        for i in range(num_bodies):
            pos = positions[i, frame]
            trails[i].append(pos)
            if len(trails[i]) > trail_length:
                trails[i].pop(0)

            # Draw planet
            glPushMatrix()
            glColor3f(*colors[i % len(colors)])
            glTranslatef(*pos)
            quad = gluNewQuadric()
            gluSphere(quad, 2.0, 10, 10)
            glPopMatrix()

        frame += 1

    draw_trails()

    glfw.swap_buffers(window)
    glfw.wait_events_timeout(frame_delay)

glfw.terminate()
